In [1]:
import pandas as pd 
import numpy as np 
import os 
import seaborn as sns 
import matplotlib.pyplot as plt
import math
import scipy.signal as sig

In [2]:
# Set input path to run all analysis 
# folder with all raw pose data (dir_in_path) 
raw_pose_data_in_path = r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_home_outputs_004'
# fps for zeno = 30
fps = 30

output_folder = r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\test_velocity_01302025\home_all_height_proxy_outputs'
output_plots = r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\test_velocity_01302025\home_all_height_proxy_outputs\pix_height_plots'
csv_file_name = 'home_depth_proxy.csv'

In [3]:
def calc_pix_size_change(start_frame, end_frame, df, fps): 
    start_row = df.loc[df['frame'] == start_frame]
    end_row = df.loc[df['frame'] == end_frame]

    # height in pixel at start and end of time group 
    pix_h_start = start_row['approx_height_Y_pix_smooth'].iloc[0]
    pix_h_end = end_row['approx_height_Y_pix_smooth'].iloc[0]

    # for consistency, only calculate when walking toward camera - height is getting bigger as walking toward 
    if pix_h_end > pix_h_start:
        # Change in pixel height from start to end 
        delta_pix_h = pix_h_end - pix_h_start

        # change in pixel height relative to start
        delta_pix_h_rel = delta_pix_h / pix_h_start

        # logarithmic scaling 
        delta_pix_h_log = math.log(pix_h_end / pix_h_start)

    else: 
        delta_pix_h = np.nan
        delta_pix_h_rel = np.nan
        delta_pix_h_log = np.nan


    return delta_pix_h, delta_pix_h_rel, delta_pix_h_log

In [4]:
# save list of all mediapipe world csv files in raw pose data folders 

raw_data_full_path_all = [] 
raw_data_file_names_all = []

depth_proxy_describe_all = pd.DataFrame()

# loop through all files in input path 
for (dir_path, dir_names, file_names) in os.walk(raw_pose_data_in_path):
    for file_name in file_names: 
        name, ext = os.path.splitext(file_name)
        ext = ext.lower()[1:]
        current_raw_data_in_path = os.path.join(dir_path, file_name) # full path to files 

        # get 
        parent_path_1, current_file_name = os.path.split(current_raw_data_in_path)
        parent_path_2, current_raw_data_folder = os.path.split(parent_path_1)
        parent_path_3, current_date = os.path.split(parent_path_2)
        parent_path_4, current_id = os.path.split(parent_path_3)
        current_id_date = os.path.join(current_id, current_date)

        # select file paths with yolo data ----------------------------------------
        if (ext == 'csv') &  ('000_raw_pose_data' in current_raw_data_in_path) & ('yolo.csv' in current_raw_data_in_path): # from run.py, outputs all saved in 000_raw_pose_data_folder
            print(current_raw_data_in_path)
            yolo_df = pd.read_csv(current_raw_data_in_path, index_col = 0)

           # pixel height -------------------------------------------------

            # calculate height in pixels - left eye to left ankle 
            l_eye_yolo_df = yolo_df.loc[yolo_df['label'] == 'left_eye']
            l_eye_yolo_df.set_index('frame', inplace=True)
            l_ankle_yolo_df = yolo_df.loc[yolo_df['label'] == 'left_ankle']
            l_ankle_yolo_df.set_index('frame', inplace=True)

            l_ankle_yolo_df = l_ankle_yolo_df.copy()
            l_ankle_yolo_df.loc[:, 'approx_height_Y_pix'] = abs(l_eye_yolo_df['Y'] - l_ankle_yolo_df['Y'])
            l_ankle_yolo_df.loc[:, 'approx_height_Y_pix_smooth'] = l_ankle_yolo_df['approx_height_Y_pix'].rolling(window=25, min_periods = 1).mean()
            
            # drop l ankle specific columns 
            height_df = l_ankle_yolo_df.drop(columns = ['X', 'Y', 'label'])
            height_df.reset_index(inplace=True)

            # add "time_group" label column, will use to group 
            height_df['time_group'] = height_df['frame'] / (fps)
            
            height_df['time_group'] = height_df['time_group'].apply(math.floor)
            # identify peaks and valleys in pixel height (approximate turn locations) 
            peaks, _ = sig.find_peaks(height_df['approx_height_Y_pix_smooth'], distance = fps)
            valleys, _ = sig.find_peaks(-height_df['approx_height_Y_pix_smooth'], distance = fps)
            peak_frames = height_df.iloc[peaks]['frame']
            valley_frames = height_df.iloc[valleys]['frame']
            peaks_valleys = np.concatenate((peak_frames, valley_frames))

            # plot height in pixels with time groups and peaks valleys 
            sns.scatterplot(x = 'frame', y = 'approx_height_Y_pix_smooth', data = height_df, label = 'approx_height_Y_pix_smooth')
            plt.plot(height_df['frame'], height_df['approx_height_Y_pix_smooth'], label='Smoothed Height')
            plt.scatter(peak_frames, height_df.iloc[peaks]['approx_height_Y_pix_smooth'], color='red', label='Peaks')
            plt.scatter(valley_frames, height_df.iloc[valleys]['approx_height_Y_pix_smooth'], color='blue', label='Valleys')
            for time_group in height_df['time_group'].unique(): 
                plt.axvline(x = time_group * (fps), color = 'grey', alpha = 0.5)
            plt.title(current_id_date + ' ' + name)

            fig_path = os.path.join(output_plots, current_id + '_' + current_date + '_' + name + '_pix_height.png') 
            plt.savefig(fig_path) 
            plt.close()

            # calculate depth proxy values and summarize 
            depth_proxies_all = []

            for current_sec_group in height_df['time_group'].unique():
                current_height_df = height_df.loc[height_df['time_group'] == current_sec_group]

                current_start_frame = current_height_df['frame'].iloc[0]
                current_end_frame = current_height_df['frame'].iloc[-1]

                # if this current time_group group contains peak or valley, skip because likeley a turn 
                if current_height_df.loc[current_start_frame: current_end_frame, 'frame'].isin(peaks_valleys).any():
                   # print('Skipped - contains peak or valley') 
                    delta_pix_h = np.nan
                    delta_pix_h_rel = np.nan
                    delta_pix_h_log = np.nan
                else: 
                    delta_pix_h, delta_pix_h_rel, delta_pix_h_log  = calc_pix_size_change(current_start_frame, current_end_frame, height_df, fps)
                
                # combine all time groups into one array 
                depth_proxies_all.append({'start_frame' : current_start_frame,
                                         'delta_pix_h' : delta_pix_h, 
                                         'delta_pix_h_rel' : delta_pix_h_rel,
                                         'delta_pix_h_log' : delta_pix_h_log})

                             
            # convert array to df 
            depth_proxies_all_df = pd.DataFrame(depth_proxies_all)
            depth_proxies_all_df =depth_proxies_all_df.replace([np.inf, -np.inf], np.nan) # replace inf with nan 

            depth_proxies_all_df = depth_proxies_all_df.drop(columns = ['start_frame'])    
            # convert to one row 
            depth_proxy_describe = depth_proxies_all_df.describe().unstack().to_frame().T
            depth_proxy_describe.columns = ['_'.join(col).strip() for col in depth_proxy_describe.columns]
            depth_proxy_describe = round(depth_proxy_describe, 2)
            depth_proxy_describe['id_video'] = current_id 
            depth_proxy_describe['visit_date_video'] = current_date
            depth_proxy_describe['filename'] = current_file_name
            
            # save all height and pixel proxies 
            depth_proxy_describe_all = pd.concat([depth_proxy_describe_all, depth_proxy_describe])   

C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_home_outputs_004\BW-0018\10-24-23\000_raw_pose_data\gait_vertical_left_yolo.csv
C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_home_outputs_004\BW-0018\10-24-23\000_raw_pose_data\gait_vertical_right_yolo.csv


C:\Users\mmccu\AppData\Local\Temp\ipykernel_16912\802653250.py:15: RuntimeWarning: divide by zero encountered in scalar divide
  delta_pix_h_rel = delta_pix_h / pix_h_start
C:\Users\mmccu\AppData\Local\Temp\ipykernel_16912\802653250.py:18: RuntimeWarning: divide by zero encountered in scalar divide
  delta_pix_h_log = math.log(pix_h_end / pix_h_start)


C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_home_outputs_004\BW-0023\05-05-23\000_raw_pose_data\gait_vertical_left_yolo.csv
C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_home_outputs_004\BW-0023\05-05-23\000_raw_pose_data\gait_vertical_right_yolo.csv
C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_home_outputs_004\BW-0023\10-23-23\000_raw_pose_data\gait_vertical_left_yolo.csv
C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_home_outputs_004\BW-0023\10-23-23\000_raw_pose_data\gait_vertical_right_yolo.csv
C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_home_outputs_004\BW-0025\11-28-2023\000_raw_pose_data\gait_vertical_left_2_yolo.csv


C:\Users\mmccu\AppData\Local\Temp\ipykernel_16912\802653250.py:15: RuntimeWarning: divide by zero encountered in scalar divide
  delta_pix_h_rel = delta_pix_h / pix_h_start
C:\Users\mmccu\AppData\Local\Temp\ipykernel_16912\802653250.py:18: RuntimeWarning: divide by zero encountered in scalar divide
  delta_pix_h_log = math.log(pix_h_end / pix_h_start)


C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_home_outputs_004\BW-0025\11-28-2023\000_raw_pose_data\gait_vertical_left_yolo.csv


...

C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_home_outputs_004\BW-0262\12-01-2023\000_raw_pose_data\gait_vertical_left_yolo.csv
C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_home_outputs_004\BW-0262\12-01-2023\000_raw_pose_data\gait_vertical_right_yolo.csv
C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_home_outputs_004\BW-0265\12-03-2023\000_raw_pose_data\gait_vertical_left_yolo.csv
C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_home_outputs_004\BW-0265\12-03-2023\000_raw_pose_data\gait_vertical_right_yolo.csv
C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_home_outputs_004\BW-0271\01-09-2024\000_raw_pose_data\gait_vertical_left_yolo.csv
C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_home_outputs_004\BW-0271\01-09-2024\000_raw_pose_data\gait_vertical_right_yolo.csv
C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_

C:\Users\mmccu\AppData\Local\Temp\ipykernel_16912\802653250.py:15: RuntimeWarning: divide by zero encountered in scalar divide
  delta_pix_h_rel = delta_pix_h / pix_h_start
C:\Users\mmccu\AppData\Local\Temp\ipykernel_16912\802653250.py:18: RuntimeWarning: divide by zero encountered in scalar divide
  delta_pix_h_log = math.log(pix_h_end / pix_h_start)


C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_home_outputs_004\BW-0276\01-17-2024\000_raw_pose_data\gait_vertical_right_yolo.csv


C:\Users\mmccu\AppData\Local\Temp\ipykernel_16912\802653250.py:15: RuntimeWarning: divide by zero encountered in scalar divide
  delta_pix_h_rel = delta_pix_h / pix_h_start
C:\Users\mmccu\AppData\Local\Temp\ipykernel_16912\802653250.py:18: RuntimeWarning: divide by zero encountered in scalar divide
  delta_pix_h_log = math.log(pix_h_end / pix_h_start)


C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_home_outputs_004\BW-0288\02-13-2024\000_raw_pose_data\gait_vertical_left_yolo.csv
C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_home_outputs_004\BW-0288\02-13-2024\000_raw_pose_data\gait_vertical_right_yolo.csv
C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_home_outputs_004\BW-0292\02-22-2024\000_raw_pose_data\gait_vertical_left_yolo.csv
C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_home_outputs_004\BW-0292\02-22-2024\000_raw_pose_data\gait_vertical_right_yolo.csv


In [5]:
depth_proxy_describe_all

,delta_pix_h_count,delta_pix_h_mean,delta_pix_h_std,delta_pix_h_min,delta_pix_h_25%,delta_pix_h_50%,delta_pix_h_75%,delta_pix_h_max,delta_pix_h_rel_count,delta_pix_h_rel_mean,...,delta_pix_h_log_mean,delta_pix_h_log_std,delta_pix_h_log_min,delta_pix_h_log_25%,delta_pix_h_log_50%,delta_pix_h_log_75%,delta_pix_h_log_max,id_video,visit_date_video,filename
0,10.0,537.28,603.55,8.94,103.34,191.98,972.64,1739.00,10.0,6.39,...,1.03,1.35,0.01,0.13,0.22,1.66,3.49,BW-0018,10-24-23,gait_vertical_left_yolo.csv
0,8.0,580.01,550.91,46.71,189.45,411.36,787.36,1734.39,7.0,3.66,...,0.76,1.08,0.06,0.26,0.40,0.59,3.17,BW-0018,10-24-23,gait_vertical_right_yolo.csv
0,15.0,266.57,160.05,29.62,131.34,210.75,386.10,511.51,15.0,0.38,...,0.32,0.14,0.08,0.25,0.29,0.38,0.54,BW-0023,05-05-23,gait_vertical_left_yolo.csv
0,16.0,260.24,155.89,35.98,158.36,246.05,381.04,547.61,16.0,0.35,...,0.29,0.13,0.10,0.17,0.34,0.39,0.50,BW-0023,05-05-23,gait_vertical_right_yolo.csv
0,13.0,62.66,47.20,10.50,20.27,52.95,94.89,147.78,13.0,0.35,...,0.29,0.13,0.06,0.20,0.34,0.38,0.47,BW-0023,10-23-23,gait_vertical_left_yolo.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,6.0,263.24,162.37,62.44,147.19,258.63,373.37,478.11,4.0,6.58,...,1.36,1.28,0.24,0.55,1.03,1.84,3.14,BW-0276,01-17-2024,gait_vertical_right_yolo.csv
0,13.0,72.25,42.34,19.09,38.45,76.17,88.02,159.47,13.0,0.39,...,0.32,0.09,0.18,0.26,0.34,0.38,0.45,BW-0288,02-13-2024,gait_vertical_left_yolo.csv
0,21.0,59.19,47.46,0.61,11.28,62.70,91.97,172.28,21.0,0.37,...,0.28,0.25,0.00,0.12,0.24,0.31,0.99,BW-0288,02-13-2024,gait_vertical_right_yolo.csv
0,7.0,225.17,215.54,12.81,117.29,181.49,233.42,680.46,7.0,0.34,...,0.28,0.18,0.01,0.23,0.29,0.30,0.61,BW-0292,02-22-2024,gait_vertical_left_yolo.csv


In [6]:
depth_proxy_describe_all.to_csv(os.path.join(output_folder, csv_file_name)) 